# PPD dos dados para treinamento, validação e teste

In [1]:
##importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random
from numpy import median, mean
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

* Outlier =  1 normal
* Outlier = -1 anormal

## IMPORTANDO dados treinamento, validacao e teste

In [2]:
# atributo DURACAO

In [3]:
## importando dataset TREINAMENTO
dfc_trein = pd.read_csv('dataseteh/pp4/eh-unifatrib-trein.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(dfc_trein.shape)
dfc_trein.head()

(126, 21)


,dia,dia_semana,fim_semana,cclothes_freq,eating_freq,mcoffe_freq,mhotfood_freq,toileting_freq,whandface_freq,cclothes_dur,...,mcoffe_dur,mhotfood_dur,toileting_dur,whandface_dur,cclothes_dfreq,eating_dfreq,mcoffe_dfreq,mhotfood_dfreq,toileting_dfreq,whandface_dfreq
0,1,0,0,3.0,2.0,4.0,1.0,6.0,10.0,1319.0,...,1600.0,2684.0,2840.0,2971.0,439.666667,2203.500000,400.000000,2684.0,473.333333,297.100000
1,2,1,0,3.0,3.0,5.0,2.0,4.0,10.0,1242.0,...,2122.0,4710.0,1937.0,2659.0,414.000000,2469.666667,424.400000,2355.0,484.250000,265.900000
2,3,2,0,3.0,2.0,3.0,2.0,7.0,8.0,1513.0,...,1157.0,6173.0,2800.0,2359.0,504.333333,1922.500000,385.666667,3086.5,400.000000,294.875000
3,4,3,0,2.0,3.0,3.0,2.0,5.0,11.0,959.0,...,1400.0,5820.0,2401.0,2862.0,479.500000,2336.666667,466.666667,2910.0,480.200000,260.181818
4,5,4,0,2.0,3.0,2.0,2.0,3.0,4.0,947.0,...,846.0,5823.0,1429.0,1066.0,473.500000,2675.666667,423.000000,2911.5,476.333333,266.500000


In [4]:
## importando dataset de TESTE
df_test2035 = pd.read_csv('dataseteh/pp4/eh-unifatrib-test-2035.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test2035.shape)
df_test2035.head()

df_test4055 = pd.read_csv('dataseteh/pp4/eh-unifatrib-test-4055.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test4055.shape)
df_test4055.head()

df_test6075 = pd.read_csv('dataseteh/pp4/eh-unifatrib-test-6075.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test6075.shape)
df_test6075.head()

df_test8095 = pd.read_csv('dataseteh/pp4/eh-unifatrib-test-8095.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test8095.shape)
df_test8095.head()

df_test2095 = pd.read_csv('dataseteh/pp4/eh-unifatrib-test-2095.csv',delimiter=';',decimal=',',thousands=".",encoding='ANSI')
print(df_test2095.shape)
df_test2095.head()

(70, 41)
(70, 41)
(70, 41)
(70, 41)
(70, 41)


,dia,dia_semana,fim_semana,cclothes_freq,eating_freq,mcoffe_freq,mhotfood_freq,toileting_freq,whandface_freq,var_cclothes_freq,...,toileting_dfreq,whandface_dfreq,var_cclothes_dfreq,var_eating_dfreq,var_mcoffe_dfreq,var_mhotfood_dfreq,var_toileting_dfreq,var_whandface_dfreq,outlier,variacao
0,1,0,0,4.0,4.0,2.0,4.0,6.0,6.0,-1,...,413.333333,259.666667,-1,-1,1,-1,1,1,-1,60-75
1,2,1,0,7.0,2.0,8.0,5.0,5.0,15.0,-1,...,440.600000,38.000000,-1,-1,-1,-1,1,-1,-1,60-75
2,3,2,0,2.0,7.0,1.0,5.0,4.0,8.0,-1,...,438.000000,219.500000,-1,-1,1,-1,-1,1,-1,60-75
3,4,3,0,3.0,4.0,3.0,1.0,13.0,19.0,1,...,84.000000,55.000000,1,-1,1,1,-1,-1,-1,60-75
4,5,4,0,2.0,7.0,2.0,2.0,12.0,4.0,1,...,69.000000,998.000000,1,-1,-1,1,-1,-1,-1,60-75


## Treinando MODELOS (4) para SEMANAS (18) para DURACAO

In [5]:
# inicializando listas para armazenar as métricas

for k in range (1,6): #iterando variacoes

    for y in range (1,5): #iteracao modelos

        #pulando a primeira semana "0" semana = x+1
        csemana = []
        cvariacao = []
        cvp = []
        cvn = []
        cfp = []
        cfn = []
        cacuracia = []
        cprecisao = []
        csensibilidade = []
        cf1escore = []
        for x in range(1,19): #iterando semanas
            semana = x

            if (k == 1): #2035
                df_test  = df_test2035.iloc[:, [3,4,5,6,7,8,15,16,17,18,19,20,27,28,29,30,31,32]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
                dfc_test = df_test2035.copy()
                variacao = '20-35'

            if (k == 2): #40-55
                df_test  = df_test4055.iloc[:, [3,4,5,6,7,8,15,16,17,18,19,20,27,28,29,30,31,32]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
                variacao = '40-55'

            if (k == 3): # 60-75
                df_test  = df_test6075.iloc[:, [3,4,5,6,7,8,15,16,17,18,19,20,27,28,29,30,31,32]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
                variacao = '60-75'

            if (k == 4): #duracao 80-95
                df_test  = df_test8095.iloc[:, [3,4,5,6,7,8,15,16,17,18,19,20,27,28,29,30,31,32]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
                variacao = '80-95'  

            if (k == 5): #duracao 20-95
                df_test  = df_test2095.iloc[:, [3,4,5,6,7,8,15,16,17,18,19,20,27,28,29,30,31,32]].copy()
                df_trein = dfc_trein.iloc[0:semana*7].copy()
                df_trein = df_trein.iloc[:, [3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]]
                variacao = '20-95'                      


            #standarização
            scaler = StandardScaler().fit(df_trein)
            df_trein = scaler.transform(df_trein).copy()
            df_test = scaler.transform(df_test).copy()

            # aplicando o primeiro algoritmo o OSMV
            # fit the model
            if y == 1:
                clf = svm.OneClassSVM(nu=0.1, kernel='rbf', gamma='scale')
                algoritmo = 'ocsvm'
            if y == 2:
                clf = LocalOutlierFactor(n_neighbors=10, novelty=True)
                algoritmo = 'lof'
            if y == 3:
                clf = IsolationForest(contamination= 0.01, random_state=1)
                algoritmo = 'iforest'
            if y == 4:
                clf = EllipticEnvelope(contamination= 0.01, random_state=0, support_fraction= 0.9)
                algoritmo = 'rce'

            clf.fit(df_trein)
            y_pred_train = clf.predict(df_trein)
            y_pred_teste = clf.predict(df_test)
            n_error_train = y_pred_train[y_pred_train == -1].size

            # calculando métricas

            dfc_test['predicao'] = y_pred_teste
            dfc_test['vp'] = np.where((dfc_test['outlier'] == -1) & (dfc_test['predicao'] == -1), 1,0)
            dfc_test['vn'] = np.where((dfc_test['outlier'] == 1) &  (dfc_test['predicao'] == 1), 1,0)
            dfc_test['fp'] = np.where((dfc_test['outlier'] == -1) & (dfc_test['predicao'] == 1), 1,0)
            dfc_test['fn'] = np.where((dfc_test['outlier'] == 1) &  (dfc_test['predicao'] == -1), 1,0)
            VP = dfc_test['vp'].sum()
            VN = dfc_test['vn'].sum()
            FP = dfc_test['fp'].sum()
            FN = dfc_test['fn'].sum()


            erro = (FP+FN)/(VP+VN+FP+FN) #erro
            acuracia =  (VP+VN)/(VP+VN+FP+FN) # accuracy
            precisao = (VP)/(FP+VP) # precision
            sensibilidade = VP/(FN+VP) #recall
            f1escore = 2 * ((precisao * sensibilidade)/(precisao + sensibilidade))
            #armazenando metricas temp
            csemana.append(semana)
            cvariacao.append(variacao)
            cvp.append(VP)
            cvn.append(VN)
            cfp.append(FP)
            cfn.append(FN)
            cacuracia.append(acuracia)
            cprecisao.append(precisao)
            csensibilidade.append(sensibilidade)
            cf1escore.append(f1escore)
            # dentro do for X SEMANAS

        #criando um dataframe a partir das métricas    
        zipped = list(zip(csemana,cvariacao,cvp,cvn,cfp,cfn,cacuracia, cprecisao, csensibilidade, cf1escore))    
        dfmetricas = pd.DataFrame(zipped, columns=['semanas','variacao','vp', 'vn', 'fp', 'fn','acuracia','precisao','sensibilidade','f1score'])
        dfmetricas['algoritmo'] = algoritmo
        dfmetricas['variacao'] = variacao
        dfmetricas= dfmetricas.replace(np.nan,0)
        #print(dfmetricas.head(20))
        #exportando dataframes para datasets

        dfmetricas.to_csv(r'dataseteh/results/ehmetricas-unifatrib-'+algoritmo+'-var-'+variacao+'-trein18.csv', sep=';', index=False, decimal=',')

        #dentro do for Y MODELOS
    #dentro do for Z ATRIBUTOS

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_lof.py:284: UserWarning: n_neighbors (10) is greater than the total number of samples (7). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:183: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-469.467694440981006 > -469.896710865013631). You may want to try with a higher value of support_fraction (current value: 0.857).
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:738: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_lof.py

In [6]:
print('DONE')

DONE
